<a href="https://colab.research.google.com/github/AyozeGS/IABD/blob/main/7RO/UT1/T2/7RO_AG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TAREA ALGORITMOS EVOLUTIVOS PROGRAMACIÓN EN IA
**EL PROBLEMA DEL VIAJANTE DE COMERCIO**

Crea un programa en Python que resuelva el problema para al menos 5 poblaciones (por ejemplo Telde, Gáldar, etc) usando el algoritmo evolutivo cuyas funciones de mutación y de recombinación vienen descritas en https://www.youtube.com/watch?v=3Kzj2FNaua8

**ENLACES DE INTERÉS:**

https://pythondiario.com/2018/05/el-problema-de-las-n-reinas-algoritmos.html

https://jarroba.com/algoritmos-geneticos-ejemplo

https://es.wikipedia.org/wiki/Problema_del_viajante

**RÚBRICA:**



*   (4 Puntos) El algoritmo muestra soluciones aceptables (para ello usa distancias reales en la matriz de distancias)
*   (4 Puntos) Se crean y comentan los métodos de mutación, fitness, y recombinanción.
*   (2 Puntos) Uso del vocabulario correcto (población, generación etc) así como explicación de los hiperparámetos (probabilidad de recombinación, de mutación, población inicial etc).

Se subirá un archivo PDF con el notebook de Jupiter incluyendo la ejecución y los markdown con las explicaciones.

In [1]:
import pandas as pd
import numpy as np
import random
from tabulate import tabulate

#Load file and clean data
df = pd.read_csv("data_municipios.csv", index_col=0)
df[df.isna()] = 0
display(df)

,Las Palmas,Telde,Arucas,Galdar,Teror,Agaete,Mogan
Distancia(km),,,,,,,
Las Palmas,0.0,15.0,9.0,22.0,13.0,26.0,39.0
Telde,15.0,0.0,17.0,28.0,15.0,30.0,33.0
Arucas,9.0,17.0,0.0,13.0,7.0,18.0,33.0
Galdar,22.0,28.0,13.0,0.0,14.0,7.0,30.0
Teror,13.0,15.0,7.0,14.0,0.0,16.0,26.0
Agaete,26.0,30.0,18.0,7.0,16.0,0.0,24.0
Mogan,39.0,33.0,33.0,30.0,26.0,24.0,0.0


### Functions

In [2]:
# Fitness: function to evaluate solution. It uses sum of distances
def fitness(member):

  sum = 0
  for i in range(len(member)-1):
    sum += df.iloc[member[i]][member[i+1]]
  return sum

In [3]:
# Mutation: function to apply a little change in a member in order to create other
def mutate(member):

  # Get 2 randoms index for exchange their values
  mutated_member = np.copy(member)
  mutated_index_1 = random.randint(0, len(member)-1)
  mutated_index_2 = random.randint(0, len(member)-1)
  while mutated_index_2 == mutated_index_1:
    mutated_index_2 = random.randint(0, len(member)-1)

  # Exchange values
  mutated_member[[mutated_index_1, mutated_index_2]] = mutated_member[[mutated_index_2, mutated_index_1]]
  return mutated_member

def mutate_generation(member, n):

  mutated_members = []
  mutated_members.append(mutate(member))

  while len(mutated_members) < n:
    mutated_member = mutate(member)
    #Check if member already in population
    if not (mutated_member == mutated_members).all(axis=1).any():
      mutated_members.append(mutated_member)
  return mutated_members

In [83]:
# Recombination : function to combine two members in order to create a new one
def recombinate(member1, member2):

  recombinated_member = np.copy(member1)

  for idx, v in enumerate(recombinated_member):
    recombinated_member[idx] = member2[member1[idx]]

  #visualize(np.concatenate(([member1], [member2], [recombinated_member]),axis=0))

  return recombinated_member

# Function to generate multiple recombinated members
def recombinate_members(members):
  recombinated_members = []

  for member1 in members:
    for member2 in members:
      if not (member1 == member2).all():
         recombinated_members.append(recombinate(member1, member2))

  return recombinated_members

In [69]:
# Selection: Choice best members to use in next generation
def selection(population, size):
  population = np.array(sorted(population, key=fitness))
  return population[:size]

In [70]:
# Visualize
def visualize(members):
  data = []
  for idx, member in enumerate(members):
    data.append([idx+1,
                 member,
                 fitness(member),
                 [df.iloc[member[i]][member[i+1]] for i in range(len(member)-1)],
                 " - ".join([df.index[i] for i in member])])

  print ("\n",tabulate(data, headers=["", "Individuo", "Distancia(Km)", "Parciales(Km)", "Ruta"]),"\n")

In [71]:
# Generate n random members
def generate_random_members(n):

  #Generate N random vectors from indexes. Ex: [4 0 5 2 6 1 3]
  population = np.array([np.random.permutation(indexes)])
  while len(population) < n:
    member = np.random.permutation(indexes)
    #Check if member already in population
    if not (member == population).all(axis=1).any():
      population = np.append(population, [member], axis=0)
  return population

# Gennerate first population
def generate_first_generation(n_population):
  return generate_random_members(n_population)


## Params

In [126]:
# Distances matrix
distances = df.to_numpy()
# Strings array for municipalities
municipalities = df.index.to_numpy()
# Integers array for municipalities
indexes = range(len(municipalities))

#hyperparameters
n_generations = 20
n_population = 20
n_chromosomes = len(municipalities)
prob_recombination = 0.3
prob_mutation = 0.4
perc_best_members = 0.1
#vars
population = []
population_total_dist = []
data = []

size_mutation = round(n_population*prob_mutation)
size_recombination = round(n_population * prob_recombination)
size_best_members = round(n_population * perc_best_members)
size_random_members = n_population - size_mutation - size_recombination - size_best_members

best_members_size = 2
while size_recombination > 2*sum(range(1,best_members_size)): #use 2 * function triangle
  best_members_size+=1
best_members_size = max(best_members_size, size_best_members)

print("Generaciones:", n_generations)
print("Poblacion:", n_population)
print("Mutaciones:", size_mutation)
print("Recombinaciones:", size_recombination)
print("Mejores:", size_best_members)
print("Aleatorios:", size_random_members)

Generaciones: 20
Poblacion: 20
Mutaciones: 8
Recombinaciones: 6
Mejores: 2
Aleatorios: 4


### Generations

In [137]:
#Generate firt population
generation = 1
population = generate_first_generation(n_population)
print("\n*********************\n*** Generacion {:2} ***\n*********************\n".format(generation))
visualize(population)

#Select best members for next generation
best_members = selection(population, best_members_size)
print("Mejores individuos\n******************".format(generation))
visualize(best_members)

# Generate next generations
while generation < n_generations:
  generation += 1
  print("\n*********************\n*** Generacion {:2} ***\n*********************\n".format(generation))

  # Generate recombinated members
  recombinated_members = recombinate_members(best_members)
  recombinated_members = recombinated_members[:size_recombination]
  print("Recombinaciones\n***************".format(generation))
  visualize(recombinated_members)

  # Generate mutated members
  mutated_members = mutate_generation(best_members[0], size_mutation)
  print("Mutaciones\n**********".format(generation))
  visualize(mutated_members)

  # Generate random members
  random_members = generate_random_members(size_random_members)
  print("Aleatorios\n**********".format(generation))
  visualize(random_members)

  # Saved members
  print("Mejores guardados\n**********".format(generation))
  visualize(best_members[:size_best_members])

  # Join members in a population
  population = np.concatenate((np.array(recombinated_members), np.array(mutated_members), random_members, best_members[:size_best_members]),axis=0)
  #print("Todos\n***".format(generation))
  #visualize(population)

  #Select best members for next generation
  best_members = selection(population, best_members_size)
  print("Mejores individuos\n******************".format(generation))
  visualize(best_members)

print("Mejor individuo\n***************".format(generation))
visualize([best_members[0]])


*********************
*** Generacion  1 ***
*********************


     Individuo          Distancia(Km)  Parciales(Km)                         Ruta
--  ---------------  ---------------  ------------------------------------  -------------------------------------------------------------
 1  [2 1 6 0 3 5 4]              134  [17.0, 33.0, 39.0, 22.0, 7.0, 16.0]   Arucas - Telde - Mogan - Las Palmas - Galdar - Agaete - Teror
 2  [3 5 1 2 4 6 0]              126  [7.0, 30.0, 17.0, 7.0, 26.0, 39.0]    Galdar - Agaete - Telde - Arucas - Teror - Mogan - Las Palmas
 3  [1 2 3 6 0 5 4]              141  [17.0, 13.0, 30.0, 39.0, 26.0, 16.0]  Telde - Arucas - Galdar - Mogan - Las Palmas - Agaete - Teror
 4  [5 4 6 3 1 0 2]              124  [16.0, 26.0, 30.0, 28.0, 15.0, 9.0]   Agaete - Teror - Mogan - Galdar - Telde - Las Palmas - Arucas
 5  [5 6 3 4 2 1 0]              107  [24.0, 30.0, 14.0, 7.0, 17.0, 15.0]   Agaete - Mogan - Galdar - Teror - Arucas - Telde - Las Palmas
 6  [0 2 3 6 5 1 4]  

# PRUEBAS

In [67]:
2*sum(range(1,4))

12

In [101]:
a = np.array([[1, 2, 3],[4,5,6],[7,8,9]])
a[:]

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [38]:
print(np.array(recombinated_members))

print(np.array(mutated_members))

print(random_members)

np.concatenate((np.array(recombinated_members), np.array(mutated_members), random_members),axis=0)

[[0 5 6 1 2 3 4]
 [3 0 4 1 5 2 6]]
[[6 1 4 3 5 0 2]
 [6 1 0 3 2 4 5]
 [6 1 2 3 4 0 5]
 [1 6 4 3 2 0 5]
 [4 1 6 3 2 0 5]
 [6 1 4 3 0 2 5]]
[[6 1 0 5 3 4 2]
 [2 4 5 6 0 1 3]]


array([[0, 5, 6, 1, 2, 3, 4],
       [3, 0, 4, 1, 5, 2, 6],
       [6, 1, 4, 3, 5, 0, 2],
       [6, 1, 0, 3, 2, 4, 5],
       [6, 1, 2, 3, 4, 0, 5],
       [1, 6, 4, 3, 2, 0, 5],
       [4, 1, 6, 3, 2, 0, 5],
       [6, 1, 4, 3, 0, 2, 5],
       [6, 1, 0, 5, 3, 4, 2],
       [2, 4, 5, 6, 0, 1, 3]])

### First Population

In [ ]:
############
# CON LIST #
############
#Generate N random vectors from indexes. Ex: [4 0 5 2 6 1 3]
while len(population) < n_population:
  member = np.random.permutation(indexes)
  #Check if member already in population
  if not population or not (member == population).all(axis=1).any():
    population.append(member)
population = np.array(population)

#############
# CON NUMPY #
#############
#Generate N random vectors from indexes. Ex: [4 0 5 2 6 1 3]
population = np.array([np.random.permutation(indexes)])
while len(population) < n_population:
  member = np.random.permutation(indexes)
  #Check if member already in population
  if not (member == population).all(axis=1).any():
    population = np.append(population, [member], axis=0)

print("\n*********************\n*** Generacion {:2} ***\n*********************\n".format(generation))
visualize(population)

#Select best members for next generation
best_members = selection(population, 2)
print("Mejores individuos\n******************".format(generation))
visualize(best_members)

In [ ]:
print(population)

population = np.array(sorted(population, key= fitness))

population_total_dist = []
population_total_dist = [fitness(member) for member in population]

print("distancias totales (km): ", population_total_dist)
print("distancia minima (km):", min(population_total_dist))


min_index = population_total_dist.index(min(population_total_dist))
best_member = population[min_index]

print("distancias parciales (km):", [df.iloc[best_member[i]][best_member[i+1]] for i in range(len(best_member)-1)])
print("mejor individuo:", best_member)
print("mejor ruta:",[df.index[i] for i in best_member])

[[3 6 5 2 4 1 0]
 [5 2 3 0 4 1 6]
 [3 4 1 2 0 6 5]
 [2 5 3 1 4 0 6]
 [3 2 4 0 5 1 6]
 [3 6 4 5 1 2 0]
 [5 1 2 3 6 4 0]
 [4 2 1 6 0 5 3]
 [0 6 1 3 5 2 4]
 [2 5 1 6 4 3 0]]
distancias totales (km):  [109.0, 114.0, 118.0, 120.0, 122.0, 128.0, 129.0, 129.0, 132.0, 143.0]
distancia minima (km): 109.0
distancias parciales (km): [30.0, 24.0, 18.0, 7.0, 15.0, 15.0]
mejor individuo: [3 6 5 2 4 1 0]
mejor ruta: ['Galdar', 'Mogan', 'Agaete', 'Arucas', 'Teror', 'Telde', 'Las Palmas']


In [ ]:
#Prueba de vector coincidiendo con fila de array
a = np.array([[5, 6, 3, 2, 1, 4, 0],
 [6, 5, 3, 4, 1, 0, 2],
 [5, 4, 1, 0, 3, 6, 2],
 [0, 3, 1, 4, 2, 6, 5],
 [0, 1, 4, 3, 6, 5, 2],
 [3, 2, 5, 6, 1, 0, 4],
 [3, 2, 6, 0, 4, 1, 5],
 [2, 0, 1, 4, 5, 6, 3],
 [1, 4, 2, 3, 0, 6, 5],
 [2, 1, 3, 4, 6, 5, 0]])
b = np.array([3, 2, 5, 6, 0, 1, 4])
c = np.array([3, 2, 5, 6, 1, 0, 4])

#print(np.any(member == np.all(population)))
print((a == b).all(axis=0))
print((a == b).all(axis=1).any())
print((a == c).all(axis=1))
print((a == c).all(axis=1).any())
print((a == c).all(axis=1).tolist().index(True))

False
False
[False False False False False  True False False False False]
True
5
